In [18]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import librosa
import glob
import IPython.display as ipd
import librosa.display
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook as tqdm
from tqdm.notebook import tqdm_notebook
from skimage.io import imread, imshow


from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score , classification_report , confusion_matrix  ,roc_curve , roc_auc_score , accuracy_score
from sklearn.metrics import precision_recall_curve , auc ,f1_score , plot_confusion_matrix , precision_score , recall_score
from sklearn.datasets import load_digits

import tensorflow
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import normalize
from tensorflow.keras.utils import to_categorical

In [19]:
file_name = []
labels = []
for i in glob.glob('/kaggle/input/ravdess-emotional-speech-audio/*'+'*/*.wav'):
    labels.append(i.split('-')[-1][:2])
    file_name.append(i)

In [20]:
df = pd.DataFrame({'FileName':file_name,'Label':labels})
df

,FileName,Label
0,/kaggle/input/ravdess-emotional-speech-audio/A...,02
1,/kaggle/input/ravdess-emotional-speech-audio/A...,02
2,/kaggle/input/ravdess-emotional-speech-audio/A...,02
3,/kaggle/input/ravdess-emotional-speech-audio/A...,02
4,/kaggle/input/ravdess-emotional-speech-audio/A...,02
...,...,...
1435,/kaggle/input/ravdess-emotional-speech-audio/A...,07
1436,/kaggle/input/ravdess-emotional-speech-audio/A...,07
1437,/kaggle/input/ravdess-emotional-speech-audio/A...,07
1438,/kaggle/input/ravdess-emotional-speech-audio/A...,07


In [21]:
# df=df.sample(10).reset_index(drop=True)
# df

In [22]:
# extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
# # extracted_features_df=pd.DataFrame({'feature':})
# extracted_features_df.head()

In [23]:
hop_length = 512
n_fft = 255

def get_features(df_in):
    features=[] 
    labels=[]
    for ind,row in tqdm(df_in.iterrows(),smoothing = 0.9, desc = "In progress"):
        y, sr = librosa.load(row[0],sr=16000, res_type='kaiser_fast')
        data = librosa.feature.mfcc(y,hop_length=hop_length,n_fft=n_fft,center=True,n_mfcc=32)
        data =data.T

        features.append((np.asarray(data)))
        labels.append(row[1])
    return features,labels
X,y=get_features(df)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


In progress: 0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -1.6336540e-05
  3.3458277e-06  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  if __name__ == "__main__":
/opt/conda/lib/python3.7/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  8.0291284e-07
 -6.4012625e-07  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  if __name__ == "__main__":
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+0

**Padding dataset**

In [25]:
import tensorflow as tf

In [26]:
X = np.asarray(X)
X = tf.keras.preprocessing.sequence.pad_sequences(X)

**Normalizing dataset**

In [27]:
X = np.array((X-np.min(X))/(np.max(X)-np.min(X)))
X = X/np.std(X)

In [28]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [29]:
X[0].shape,X[1].shape,X[2].shape

((165, 32), (165, 32), (165, 32))

**MODEL RNN**

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from keras.layers import LeakyReLU
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

In [31]:
np.array(x_train).shape

(1152, 165, 32)

In [44]:
input_shape= (165,32)
model = Sequential()
# model.add(LSTM(1024,input_shape=input_shape))
# model.add(Dropout(0.2))
model.add(LSTM(128,input_shape=input_shape))
# model.add(Dense(512, activation='relu'))
# model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(48, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(25, activation='softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               82432     
_________________________________________________________________
dense_11 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_12 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_13 (Dense)             (None, 48)                3120      
_________________________________________________________________
dropout_1 (Dropout)          (None, 48)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 25)                1225      
Total params: 136,249
Trainable params: 136,249
Non-trainable params: 0
________________________________________________

In [45]:
model.compile(loss='SparseCategoricalCrossentropy', metrics=['accuracy'], optimizer='adam')

In [47]:
yy =[]
for i in y_train:
    yy.append(int(i))

In [48]:
from keras.callbacks import ModelCheckpoint 
checkpointer = ModelCheckpoint(filepath='./audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

h = model.fit(x_train, np.array(yy), epochs=50, batch_size=32,  callbacks=[checkpointer],shuffle=True,verbose=1)

print("Training completed in time: ", datetime.now() - start)

Epoch 1/50
36/36 [==============================] - 7s 137ms/step - loss: 3.2258 - accuracy: 0.0408
Epoch 2/50
36/36 [==============================] - 5s 152ms/step - loss: 3.2147 - accuracy: 0.0451
Epoch 3/50
36/36 [==============================] - 5s 139ms/step - loss: 3.2088 - accuracy: 0.0408
Epoch 4/50
36/36 [==============================] - 6s 157ms/step - loss: 3.2093 - accuracy: 0.0425
Epoch 5/50
36/36 [==============================] - 5s 140ms/step - loss: 3.2036 - accuracy: 0.0399
Epoch 6/50
36/36 [==============================] - 5s 138ms/step - loss: 3.1995 - accuracy: 0.0495
Epoch 7/50
36/36 [==============================] - 5s 137ms/step - loss: 3.1953 - accuracy: 0.0477
Epoch 8/50
36/36 [==============================] - 5s 139ms/step - loss: 3.1970 - accuracy: 0.0365
Epoch 9/50
36/36 [==============================] - 5s 135ms/step - loss: 3.1962 - accuracy: 0.0373
Epoch 10/50
36/36 [==============================] - 6s 158ms/step - loss: 3.1932 - accuracy: 0.0365

In [49]:
yy_test = []
for i in y_test:
    yy_test.append(int(i))

In [50]:
test_accuracy=model.evaluate(x_test,np.array(yy_test),verbose=1)
print(test_accuracy[1]*100)

9/9 [==============================] - 1s 59ms/step - loss: 3.1643 - accuracy: 0.0312
3.125
